# Neural-based classification rule learning for sequential data

## Install and Import used packages

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from src.run import train, test, test_rule

c:\Users\14153\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path_checkpoint = "peptides_checkpoint.pt"

## Parameters

### Data

In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import requests
from io import BytesIO
from zipfile import ZipFile
import os

if not os.path.exists("ACPs_Breast_cancer.csv"):
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00589/Anticancer_Peptides.zip'
    r = requests.get(url)
    z = ZipFile(BytesIO(r.content))
    df = pd.read_csv(z.open("dataset\question.csv"))
    df.to_csv("dataset\question.csv")
else:
    df = pd.read_csv("dataset\question.csv")
df

Unnamed: 0    ID                                         sequence  class
0              1     1            6  7  8  9  10  11  12  13  14  15  5      0
1              2     2                    16  17  18  19  20  21  22  5      0
2              3     3          6  7  23  24  25  26  27  28  29  30  5      0
3              4     4              6  7  31  32  33  34  35  11  36  5      0
4              5     5                         0  37  18  38  39  40  5      0
...          ...   ...                                              ...    ...
1725        1726  1726  16  2252  1146  220  3606  1  3607  11  3608  5      1
1726        1727  1727             16  1146  17  417  90  3609  3610  5      1
1727        1728  1728                        16  17  18  2065  2524  5      1
1728        1729  1729  725  383  204  888  826  177  2897  26  3611  5      1
1729        1730  1730         60  123  1739  79  480  481  3495  40  5      1

[1730 rows x 4 columns]

In [6]:
batch_size=100
val_size=0.2
test_size=0.2


In [7]:
from src.data.datasets import UCIAnticancerDataset
from src.data.util import construct_data,construct_loader
dataset = UCIAnticancerDataset(df)

# train_loader = construct_loader(dataset, train_indices, batch_size)
# five-folder cross validation
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
train_loaders = []
val_loaders = []
k = 1
for train_index, test_index in kf.split(dataset):
    print("Fold: ", k)

    # construct_loader
    train_loader = construct_loader(dataset, train_index, batch_size)
    val_loader = construct_loader(dataset, test_index, len(test_index))

    train_loaders.append(train_loader)
    val_loaders.append(val_loader)
    k += 1
    
    
    

Distrib 0.48265895953757226 835.0 1730
Fold:  1
Fold:  2
Fold:  3
Fold:  4


c:\Users\14153\Desktop\复现\cr2n-main\cr2n-main\src\data\util.py:18: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sampler = BucketBatchSampler(np.array(dataset.x, dtype=object)[indices], batch_size)


Fold:  5


### Model

In [8]:
input_size = len(dataset.features_name)
window_size = 3
base_model_hidden_size = window_size * 2
base_or_output_size=1
output_size=1

pad_border = True

max_sequence_length = max(
    train_loader.batch_sampler.max_length,
    val_loader.batch_sampler.max_length,
)

# Compute dimension of convOR input size
conv_dim_out = max_sequence_length - (window_size - 1)
if pad_border:
    conv_dim_out = conv_dim_out + (window_size - 1) * 2

### Training

In [9]:
from src.sparsify import Pruning

pruning = Pruning()
pruning30 = Pruning(start=30)
no_pruning = False

In [10]:
pruning_strategy = pruning30 # or no_pruning or pruning30
epochs = 200
# Other parameters are fixed directly in train() function

## Run Local model

In [11]:
from src.models import LocalModel
# original model
# for i in range(10):
#     local_model = LocalModel(input_size, window_size, pad_border, max_sequence_length, base_model_hidden_size, conv_dim_out, base_or_output_size, output_size)
    # train(local_model, train_loader, val_loader, pruning=pruning_strategy, epochs=epochs, path_checkpoint=path_checkpoint, verbose=True)
#     rule = local_model.extract_rule(features_names=dataset.features_name, verbose=False)
#     test(local_model.model, test_loader)
#     test_rule(rule, test_loader)
#     print('-----')

# k-fold cross validation
for i in range(5):

    local_model = LocalModel(input_size, window_size, pad_border, max_sequence_length, base_model_hidden_size, conv_dim_out, base_or_output_size, output_size)
    train(local_model, train_loaders[i], val_loaders[i], pruning=pruning_strategy, epochs=epochs, path_checkpoint=path_checkpoint, verbose=True)
    rule = local_model.extract_rule(features_names=dataset.features_name, verbose=False)
    test(local_model.model,  val_loaders[i])
    test_rule(rule,  val_loaders[i])
    print('-----')

Number of batches: 89


KeyboardInterrupt: 

In [ ]:
print(rule)

## Run Global Model

In [ ]:
from src.models import GlobalModel

for i in range(5):
    global_model = GlobalModel(input_size, window_size, pad_border, max_sequence_length, base_model_hidden_size, conv_dim_out, base_or_output_size, output_size)
    train(global_model, train_loaders[i], val_loaders[i], pruning=pruning_strategy, epochs=epochs, path_checkpoint=path_checkpoint, verbose=True)
    rule = global_model.extract_rule(features_names=dataset.features_name, verbose=False)
    test(global_model.model,  val_loaders[i])
    test_rule(rule,  val_loaders[i])
    print('-----')

In [ ]:
rule = global_model.extract_rule(features_names=dataset.features_name, verbose=False)
test(global_model.model, test_loader)
test_rule(rule, test_loader)
print('-----')
print(rule)
test_rule(rule, test_loader)